In [2]:
import os, sys
import traci

# Check if SUMO_HOME is defined 
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

In [3]:
# Step 1: Define Sumo configuration
Sumo_config = [
    'sumo-gui',
    '-c', "C:/Users/shrof/OneDrive/Documents/Documents/UVA/Capstone/2025-01-15-10-16-48/osm.sumocfg",
    '--step-length', '0.05',
    '--delay', '0',
    '--lateral-resolution', '0.1'
]

In [4]:
# Step 2: Open connection between SUMO and Traci
traci.start(Sumo_config)

(21, 'SUMO 1.21.0')

In [10]:
# Step 3: Define Variables
vehicle_speed = 0
total_speed = 0

In [ ]:
# Step 4: Define Functions


In [ ]:
# Step 8: Take simulation steps until there are no more vehicles in the network
while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep() # Move simulation forward 1 step
    # Here you can decide what to do with simulation data at each step
    if 'veh1' in traci.vehicle.getIDList():
        vehicle_speed = traci.vehicle.getSpeed('veh1')
        total_speed = total_speed + vehicle_speed
    # step_count = step_count + 1
    print(f"Vehicle speed: {vehicle_speed} m/s")

In [5]:
# Step 9: Close connection between SUMO and Traci

traci.close()

AttributeError: module 'traci' has no attribute 'shutdown'

In [1]:
import os
import sys
import random
import numpy as np
import traci
from stable_baselines3 import DQN  # Example RL algorithm
from stable_baselines3.common.vec_env import DummyVecEnv
import gym

# Step 1: Check if SUMO_HOME is defined
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

# Step 2: Define the SUMO configuration
Sumo_config = [
    'sumo-gui',  # Use 'sumo' for no GUI
    '-c', "C:/Users/shrof/OneDrive/Documents/Documents/UVA/Capstone/2025-01-29-09-00-36/osm.sumocfg",
    '--step-length', '0.05',
    '--delay', '0',
    '--lateral-resolution', '0.1'
]

# Step 3: Create Custom SUMO Environment for RL
class SumoEnv(gym.Env):

    def __init__(self):
        
        self.observation_space = np.zeros(5) # 5 features: x, y, speed, density, distance
        self.action_space = [0, 1, 2, 3]  # Example: actions for rerouting vehicles
        self.vehicle_id = "veh14"  # Example vehicle ID

        traci.start(Sumo_config)

    def reset(self):
        # Restart the simulation
        traci.close()
        traci.start(Sumo_config)
        # Reset environment and return initial observation
        return self.get_observation()

    def get_observation(self):
        
        vehicle_position = traci.vehicle.getPosition(self.vehicle_id)
        vehicle_speed = traci.vehicle.getSpeed(self.vehicle_id)
        traffic_density = len(traci.vehicle.getIDList())
        destination = traci.vehicle.getRoute(self.vehicle_id)[-1]  # Last point in route
        distance_to_target = traci.simulation.getDistance2D(
            vehicle_position[0], vehicle_position[1], *destination
        )

        # Return observation as a NumPy array
        return np.array([
            vehicle_position[0],  # x-coordinate
            vehicle_position[1],  # y-coordinate
            vehicle_speed,        # Vehicle's speed
            traffic_density,      # Number of vehicles on the road
            distance_to_target    # Distance to the destination
        ])


    def step(self, action):
        """Execute an action and return new state, reward, done, and info."""
        
        # Get the current position of the vehicle
        current_edge = traci.vehicle.getRoadID(self.vehicle_id)
        vehicle_position = traci.vehicle.getPosition(self.vehicle_id)
        
        # Get the junction (node) that the current edge leads to
        current_junction = traci.edge.getToNode(current_edge)
        
        # Retrieve outgoing edges from the junction (potential neighboring edges)
        neighboring_edges = list(traci.junction.getOutgoing(current_junction))
        
        # Select the next edge based on the action
        if action < len(neighboring_edges):
            new_edge = neighboring_edges[action]  # Selecting the next edge based on action
        else:
            new_edge = current_edge  # Stay on the same edge if action is invalid
        
        # Set a new route dynamically (only if the edge is valid)
        if new_edge != current_edge:
            traci.vehicle.setRoute(self.vehicle_id, [new_edge])

        # Advance the simulation by one step
        traci.simulationStep()

        # Gather the new observation (e.g., vehicle position, traffic density, etc.)
        observation = self.get_observation()

        # Calculate the reward
        reward = -traci.vehicle.getWaitingTime(self.vehicle_id)  # Example: minimize waiting time

        # Check if the simulation is done (e.g., vehicle reaches destination or simulation ends)
        done = traci.vehicle.getRoadID(self.vehicle_id) == traci.vehicle.getRoute(self.vehicle_id)[-1]

        # Optional: Add additional info for logging or analysis
        info = {
            "current_edge": traci.vehicle.getRoadID(self.vehicle_id),
            "current_speed": traci.vehicle.getSpeed(self.vehicle_id),
            "remaining_distance": traci.vehicle.getDistance(self.vehicle_id),
        }

        return observation, reward, done, info


    def close(self):
        
        """Ensure SUMO is closed when environment is shut down"""

        if traci.isSimulationRunning():
            traci.close()

# Step 4: Train RL Model
if __name__ == "__main__":
    # Wrap environment with stable-baselines3's DummyVecEnv
    env = DummyVecEnv([lambda: SumoEnv()])

    # Use DQN as the RL algorithm
    model = DQN("MlpPolicy", env, verbose=1)

    # Train the model
    try:
        model.learn(total_timesteps=10)
    finally:
        env.close()

    # Save the model
    model.save("sumo_rl_model")

    # # Test the model
    # obs = env.reset()
    # for _ in range(100):
    #     action, _ = model.predict(obs)
    #     obs, reward, done, _ = env.step(action)
    #     if done:
    #         break

traci.vehicle.getRoute()

#traci.edge.setMaxSpeed("I-81_section_mid", 5)  # Set speed to 5 m/s, traffic jam simulation

# Close the environment
env.close()


FatalTraCIError: Connection closed by SUMO.

In [2]:
import os
import sys
import numpy as np
import traci
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
import shimmy

# Check if SUMO_HOME is set
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")

# SUMO Configuration
Sumo_config = [
    'sumo-gui',
    '-c', "C:/Users/shrof/OneDrive/Documents/Documents/UVA/Capstone/2025-01-29-09-00-36/osm.sumocfg",
    '--step-length', '1',
    '--delay', '0',
    '--lateral-resolution', '0.1'
]

class SumoEnv(gym.Env):
    def __init__(self):
        print('init_function')
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(2)  # Example actions: change route

        self.vehicle_id = "veh0"
        print('starting SUMO')
        traci.start(Sumo_config)
        print('succesfull_started_SUMO')

    def reset(self):
        print('reset_function')
        #traci.close()
        #traci.start(Sumo_config)
        return self.get_observation()

    def get_observation(self):
        print('obs_function')
        vehicle_position = traci.vehicle.getPosition(self.vehicle_id)
        vehicle_speed = traci.vehicle.getSpeed(self.vehicle_id)
        traffic_density = len(traci.vehicle.getIDList())
        
        # destination = traci.vehicle.getRoute(self.vehicle_id)[-1]  # Last point in route

        # print('obsfunction1')
        # current_edge = traci.vehicle.getRoadID(self.vehicle_id)
        # current_position = traci.vehicle.getLanePosition(self.vehicle_id)


        # distance_to_target = traci.simulation.getDistanceRoad(
        #     current_edge, current_position, destination, 0.0, isDriving=True
        # )

        return np.array([vehicle_position[0], vehicle_position[1], vehicle_speed, traffic_density])#, distance_to_target])

    def step(self, action):
        print('step_function')
        current_edge = traci.vehicle.getRoadID(self.vehicle_id)
        all_edges = traci.vehicle.getRoute(self.vehicle_id)
        

        # Select new edge based on action
        if action < len(all_edges) & current_edge.startswith(":") == False:
            new_edge = all_edges[action]
        else:
            new_edge = current_edge

        if new_edge != current_edge:
            traci.vehicle.setRoute(self.vehicle_id, [new_edge])

        # if action == 0:
        #     # Action 0: Stay on the current node (no change in route)
        #     new_edge = current_edge
        # elif action == 1:
        #     # Action 1: Switch to a random adjacent node
        #     # Get the possible adjacent nodes
        #     adjacent_edges = self.get_adjacent_edges(current_edge)
        #     if adjacent_edges:
        #         new_edge = np.random.choice(adjacent_edges)
        #     else:
        #         new_edge = current_edge  # If no adjacent nodes, stay on current node

        traci.simulationStep()

        observation = self.get_observation()
        reward = -traci.vehicle.getWaitingTime(self.vehicle_id)
        done = traci.vehicle.getRoadID(self.vehicle_id) == traci.vehicle.getRoute(self.vehicle_id)[-1]

        return observation, reward, done, {}

    def close(self):
        print('close_function')
        #if traci.isSimulationRunning():
        traci.close()
        print('succesfully_closed')

#if __name__ == "__main__":
    # env = DummyVecEnv([lambda: SumoEnv()])
    # print('1')
    # model = DQN("MlpPolicy", env, verbose=1)
    # print('1.1')

    # model.learn(total_timesteps=40)
    # print('1.2')

    # model.save("sumo_rl_model")
    # env.close()
    # print('1.3')
    # # Test the trained model
    
    # env = SumoEnv()
    # print('1.4')
    # obs = env.reset()
    # print('1.5')
    # route_taken = []

    # for _ in range(50):
    #     action, _ = model.predict(obs)
    #     obs, reward, done, _ = env.step(action)
    #     route_taken.append(traci.vehicle.getRoadID(env.vehicle_id))
    #     if done:
    #         break

    # print("Optimized route:", route_taken)
    # env.close()


In [2]:
if __name__ == "__main__":
    env = DummyVecEnv([lambda: SumoEnv()])
    print('1')
    model = DQN("MlpPolicy", env, verbose=1)
    print('1.1')

    model.learn(total_timesteps=40)
    print('1.2')

    model.save("sumo_rl_model")
    # env.close()
    # print('1.3')
    # # Test the trained model
    
    # env = SumoEnv()
    # print('1.4')
    # obs = env.reset()
    # print('1.5')
    # route_taken = []

    # for _ in range(50):
    #     action, _ = model.predict(obs)
    #     obs, reward, done, _ = env.step(action)
    #     route_taken.append(traci.vehicle.getRoadID(env.vehicle_id))
    #     if done:
    #         break

    # print("Optimized route:", route_taken)
    # env.close()

init_function
starting SUMO
 Retrying in 1 seconds
succesfull_started_SUMO
1
Using cpu device


c:\Users\shrof\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


1.1
reset_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_funct

In [ ]:
from stable_baselines3 import DQN

# Load the saved model
model = DQN.load("sumo_rl_model")  # Load the model that was previously saved

# Initialize the environment
env = SumoEnv()
print('1.4')

# Reset the environment and get the initial observation
obs = env.reset()
print('1.5')

# List to track the route the vehicle takes
route_taken = []

# Run for 50 steps or until done
for _ in range(50):
    # Use the model to predict the action
    action, _ = model.predict(obs)
    
    # Take the action in the environment
    obs, reward, done, _ = env.step(action)
    
    # Track the route the vehicle is on
    route_taken.append(traci.vehicle.getRoadID(env.vehicle_id))
    
    # If the vehicle reaches the destination, break
    if done:
        break

# Output the optimized route
print("Optimized route:", route_taken)

# Close the environment
env.close()


init_function
starting SUMO
succesfull_started_SUMO
1.4
reset_function
obs_function
1.5
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_function
step_function
obs_fun

In [1]:
%tb

No traceback available to show.
